This is an implementation of the paper Deep Bayesian Active Learning with Image Data using PyTorch and modAL. 

modAL is an active learning framework for Python3, designed with modularity, flexibility and extensibility in mind. Built on top of scikit-learn, it allows you to rapidly create active learning workflows with nearly complete freedom. What is more, you can easily replace parts with your custom built solutions, allowing you to design novel algorithms with ease.

Since modAL only supports sklearn models, we will also use [skorch](https://skorch.readthedocs.io/en/stable/), a scikit-learn compatible neural network library that wraps PyTorch. 

In [1]:
import torch
import numpy as np
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from skorch import NeuralNetClassifier
from modAL.models import ActiveLearner

from VAE import VAE

C:\Users\saumy\Anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] The specified module could not be found
  warn(f"Failed to load image Python extension: {e}")


### architecture of the network we will be using

We will use the architecture described in the paper.

In [2]:
class CNN(nn.Module):
    def __init__(self,):
        super(CNN, self).__init__()
        self.convs = nn.Sequential(
                                nn.Conv2d(1,32,4),
                                nn.ReLU(),
                                nn.Conv2d(32,32,4),
                                nn.ReLU(),
                                nn.MaxPool2d(2),
                                nn.Dropout(0.25)
        )
        self.fcs = nn.Sequential(
                                nn.Linear(11*11*32,128),
                                nn.ReLU(),
                                nn.Dropout(0.5),
                                nn.Linear(128,10),
        )

    def forward(self, x):
        out = x
        out = self.convs(out)
        out = out.view(-1,11*11*32)
        out = self.fcs(out)
        return out

### read training data

In [3]:
mnist_train = MNIST('.', train=True, download=True, transform=ToTensor())
mnist_test  = MNIST('.', train=False,download=True, transform=ToTensor())
traindataloader = DataLoader(mnist_train, shuffle=True, batch_size=60000)
testdataloader  = DataLoader(mnist_test , shuffle=True, batch_size=10000)
X_train, y_train = next(iter(traindataloader))
X_test , y_test  = next(iter(testdataloader))
X_train, y_train = X_train.detach().cpu().numpy(), y_train.detach().cpu().numpy()
X_test, y_test = X_test.detach().cpu().numpy(), y_test.detach().cpu().numpy()

### preprocessing

In [4]:
X_train = X_train.reshape(60000, 1, 28, 28)
X_test = X_test.reshape(10000, 1, 28, 28)

### initial labelled data
We initialize the labelled set with 20 balanced randomly sampled examples

In [5]:
initial_idx = np.array([],dtype=int)
for i in range(10):
    idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
    initial_idx = np.concatenate((initial_idx, idx))

X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

### initial unlabelled pool

In [6]:
X_pool = np.delete(X_train, initial_idx, axis=0)
y_pool = np.delete(y_train, initial_idx, axis=0)

## Query Strategies

### Uniform
All the acquisition function we will use will be compared to the uniform acquisition function $\mathbb{U}_{[0,1]}$ which will be our baseline that we would like to beat.

In [7]:
def uniform(learner, X, n_instances=1):
    query_idx = np.random.choice(range(len(X)), size=n_instances, replace=False)
    return query_idx, X[query_idx]

### Entropy
Our first acquisition function is the entropy:
$$ \mathbb{H} = - \sum_{c} p_c \log(p_c)$$
where $p_c$ is the probability predicted for class c. This is approximated by:
\begin{align}
p_c &= \frac{1}{T} \sum_t p_{c}^{(t)} 
\end{align}
where $p_{c}^{t}$ is the probability predicted for class c at the t th feedforward pass.

In [8]:
def max_entropy(learner, X, n_instances=1, T=100):
    random_subset = np.random.choice(range(len(X)), size=2000, replace=False)
    with torch.no_grad():
        outputs = np.stack([torch.softmax(learner.estimator.forward(X[random_subset], training=True),dim=-1).cpu().numpy()
                            for t in range(100)])
    pc = outputs.mean(axis=0)
    acquisition = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    idx = (-acquisition).argsort()[:n_instances]
    query_idx = random_subset[idx]
    return query_idx, X[query_idx]

In [9]:
def bald(learner, X, n_instances=1, T=100):
    random_subset = np.random.choice(range(len(X)), size=2000, replace=False)
    with torch.no_grad():
        outputs = np.stack([torch.softmax(learner.estimator.forward(X[random_subset], training=True),dim=-1).cpu().numpy()
                            for t in range(100)])
    pc = outputs.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(outputs * np.log(outputs + 1e-10), axis=-1), axis=0)  # [batch size]
    acquisition = H - E_H
    idx = (-acquisition).argsort()[:n_instances]
    query_idx = random_subset[idx]
    return query_idx, X[query_idx]    

In [10]:
def save_list(input_list, name):
    with open("perf_lists/" + name, 'w') as f:
        for val in input_list:
            f.write("%s\n" % val)

### Active Learning Procedure

In [11]:
def active_learning_procedure(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=100,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
    return perf_hist

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
estimator = NeuralNetClassifier(CNN,
                                max_epochs=50,
                                batch_size=128,
                                lr=0.001,
                                optimizer=torch.optim.Adam,
                                criterion=torch.nn.CrossEntropyLoss,
                                train_split=None,
                                verbose=0,
                                device=device)
uniform_perf_hist = active_learning_procedure(uniform,
                                              X_test,
                                              y_test,
                                              X_pool,
                                              y_pool,
                                              X_initial,
                                              y_initial,
                                              estimator,)
save_list(uniform_perf_hist, "uniform_perf_hist")

Accuracy after query 1: 0.6350
Accuracy after query 2: 0.6905
Accuracy after query 3: 0.7269
Accuracy after query 4: 0.7543
Accuracy after query 5: 0.7582
Accuracy after query 6: 0.7580
Accuracy after query 7: 0.7750
Accuracy after query 8: 0.7951
Accuracy after query 9: 0.7815
Accuracy after query 10: 0.7930
Accuracy after query 11: 0.7815
Accuracy after query 12: 0.8073
Accuracy after query 13: 0.8153
Accuracy after query 14: 0.8263
Accuracy after query 15: 0.8496
Accuracy after query 16: 0.8477
Accuracy after query 17: 0.8537
Accuracy after query 18: 0.8614
Accuracy after query 19: 0.8728
Accuracy after query 20: 0.8675
Accuracy after query 21: 0.8697
Accuracy after query 22: 0.8608
Accuracy after query 23: 0.8926
Accuracy after query 24: 0.8825
Accuracy after query 25: 0.8918
Accuracy after query 26: 0.9130
Accuracy after query 27: 0.8944
Accuracy after query 28: 0.8875
Accuracy after query 29: 0.9089
Accuracy after query 30: 0.9154
Accuracy after query 31: 0.9096
Accuracy after qu

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
estimator = NeuralNetClassifier(CNN,
                                max_epochs=50,
                                batch_size=128,
                                lr=0.001,
                                optimizer=torch.optim.Adam,
                                criterion=torch.nn.CrossEntropyLoss,
                                train_split=None,
                                verbose=0,
                                device=device)
bald_perf_hist = active_learning_procedure(bald,
                                           X_test,
                                           y_test,
                                           X_pool,
                                           y_pool,
                                           X_initial,
                                           y_initial,
                                           estimator,)
save_list(bald_perf_hist, "bald_perf_hist")

Accuracy after query 1: 0.6331
Accuracy after query 2: 0.6907
Accuracy after query 3: 0.7075
Accuracy after query 4: 0.6924
Accuracy after query 5: 0.7374
Accuracy after query 6: 0.7593
Accuracy after query 7: 0.7358
Accuracy after query 8: 0.7892
Accuracy after query 9: 0.7855
Accuracy after query 10: 0.7754
Accuracy after query 11: 0.7653
Accuracy after query 12: 0.8392
Accuracy after query 13: 0.8370
Accuracy after query 14: 0.8198
Accuracy after query 15: 0.8549
Accuracy after query 16: 0.8867
Accuracy after query 17: 0.8642
Accuracy after query 18: 0.9046
Accuracy after query 19: 0.8931
Accuracy after query 20: 0.8635
Accuracy after query 21: 0.9067
Accuracy after query 22: 0.9148
Accuracy after query 23: 0.9157
Accuracy after query 24: 0.9115
Accuracy after query 25: 0.8997
Accuracy after query 26: 0.9113
Accuracy after query 27: 0.9176
Accuracy after query 28: 0.9225
Accuracy after query 29: 0.9259
Accuracy after query 30: 0.9404
Accuracy after query 31: 0.9418
Accuracy after qu

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
estimator = NeuralNetClassifier(CNN,
                                max_epochs=50,
                                batch_size=128,
                                lr=0.001,
                                optimizer=torch.optim.Adam,
                                criterion=torch.nn.CrossEntropyLoss,
                                train_split=None,
                                verbose=0,
                                device=device)
entropy_perf_hist = active_learning_procedure(max_entropy,
                                              X_test,
                                              y_test,
                                              X_pool,
                                              y_pool,
                                              X_initial,
                                              y_initial,
                                              estimator,)
save_list(entropy_perf_hist, "entropy_perf_hist")

Accuracy after query 1: 0.6666
Accuracy after query 2: 0.7164
Accuracy after query 3: 0.6892
Accuracy after query 4: 0.7094
Accuracy after query 5: 0.7170
Accuracy after query 6: 0.7287
Accuracy after query 7: 0.7863
Accuracy after query 8: 0.8058
Accuracy after query 9: 0.7940
Accuracy after query 10: 0.7868
Accuracy after query 11: 0.8003
Accuracy after query 12: 0.8124
Accuracy after query 13: 0.8188
Accuracy after query 14: 0.8533
Accuracy after query 15: 0.8176
Accuracy after query 16: 0.8485
Accuracy after query 17: 0.8672
Accuracy after query 18: 0.8745
Accuracy after query 19: 0.8816
Accuracy after query 20: 0.8969
Accuracy after query 21: 0.8860
Accuracy after query 22: 0.8821
Accuracy after query 23: 0.9158
Accuracy after query 24: 0.8903
Accuracy after query 25: 0.8863
Accuracy after query 26: 0.9109
Accuracy after query 27: 0.9157
Accuracy after query 28: 0.9199
Accuracy after query 29: 0.9324
Accuracy after query 30: 0.9346
Accuracy after query 31: 0.9283
Accuracy after qu

In [15]:
def active_learning_procedure_generative(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=100,
                              n_instances=10):
    weights_location = 'weights.pt'
    vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
    vae.load_state_dict(torch.load(weights_location))

    def vaeNewSampleGenerator(vae, samples):    
        vae.eval()
        with torch.no_grad():
            new_samples, _, _ = vae(samples.reshape(samples.shape[0], -1))
            return new_samples.reshape(samples.shape)

    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        
        new_samples = vaeNewSampleGenerator(vae, torch.tensor(X_pool[query_idx]))
        new_samples = new_samples.detach().cpu().numpy()
        new_samples = np.concatenate((new_samples, X_pool[query_idx]))
        new_labels = np.concatenate((y_pool[query_idx], y_pool[query_idx]))
        learner.teach(new_samples, new_labels)
        
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
    return perf_hist

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
estimator = NeuralNetClassifier(CNN,
                                max_epochs=50,
                                batch_size=128,
                                lr=0.001,
                                optimizer=torch.optim.Adam,
                                criterion=torch.nn.CrossEntropyLoss,
                                train_split=None,
                                verbose=0,
                                device=device)
bald_generative_perf_hist = active_learning_procedure_generative(bald,
                                           X_test,
                                           y_test,
                                           X_pool,
                                           y_pool,
                                           X_initial,
                                           y_initial,
                                           estimator,)
save_list(bald_generative_perf_hist, "bald_generative_perf_hist")

C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy after query 1: 0.6283


C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy after query 2: 0.6191


C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


...

Accuracy after query 98: 0.9721


C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy after query 99: 0.9734


C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy after query 100: 0.9744


In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
estimator = NeuralNetClassifier(CNN,
                                max_epochs=50,
                                batch_size=128,
                                lr=0.001,
                                optimizer=torch.optim.Adam,
                                criterion=torch.nn.CrossEntropyLoss,
                                train_split=None,
                                verbose=0,
                                device=device)
entropy_generative_perf_hist = active_learning_procedure_generative(max_entropy,
                                           X_test,
                                           y_test,
                                           X_pool,
                                           y_pool,
                                           X_initial,
                                           y_initial,
                                           estimator,)
save_list(entropy_generative_perf_hist, "entropy_generative_perf_hist")

C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy after query 1: 0.6873


C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy after query 2: 0.6613


C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


...

Accuracy after query 98: 0.9673


C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy after query 99: 0.9685


C:\Users\saumy\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy after query 100: 0.9705


In [18]:
def active_learning_procedure_duplicate(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=100,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        
        new_samples = np.concatenate((X_pool[query_idx], X_pool[query_idx]))
        new_labels = np.concatenate((y_pool[query_idx], y_pool[query_idx]))
        learner.teach(new_samples, new_labels)
        
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
    return perf_hist

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
estimator = NeuralNetClassifier(CNN,
                                max_epochs=50,
                                batch_size=128,
                                lr=0.001,
                                optimizer=torch.optim.Adam,
                                criterion=torch.nn.CrossEntropyLoss,
                                train_split=None,
                                verbose=0,
                                device=device)
bald_duplicate_perf_hist = active_learning_procedure_duplicate(bald,
                                           X_test,
                                           y_test,
                                           X_pool,
                                           y_pool,
                                           X_initial,
                                           y_initial,
                                           estimator,)
save_list(bald_duplicate_perf_hist, "bald_duplicate_perf_hist")

Accuracy after query 1: 0.6701
Accuracy after query 2: 0.6910
Accuracy after query 3: 0.7327
Accuracy after query 4: 0.7157
Accuracy after query 5: 0.6995
Accuracy after query 6: 0.7421
Accuracy after query 7: 0.7293
Accuracy after query 8: 0.7230
Accuracy after query 9: 0.7675
Accuracy after query 10: 0.8022
Accuracy after query 11: 0.8654
Accuracy after query 12: 0.8456
Accuracy after query 13: 0.8823
Accuracy after query 14: 0.8660
Accuracy after query 15: 0.9124
Accuracy after query 16: 0.9023
Accuracy after query 17: 0.8841
Accuracy after query 18: 0.9086
Accuracy after query 19: 0.9159
Accuracy after query 20: 0.9206
Accuracy after query 21: 0.9273
Accuracy after query 22: 0.9378
Accuracy after query 23: 0.9327
Accuracy after query 24: 0.9357
Accuracy after query 25: 0.9391
Accuracy after query 26: 0.9288
Accuracy after query 27: 0.9374
Accuracy after query 28: 0.9444
Accuracy after query 29: 0.9450
Accuracy after query 30: 0.9470
Accuracy after query 31: 0.9527
Accuracy after qu

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"
estimator = NeuralNetClassifier(CNN,
                                max_epochs=50,
                                batch_size=128,
                                lr=0.001,
                                optimizer=torch.optim.Adam,
                                criterion=torch.nn.CrossEntropyLoss,
                                train_split=None,
                                verbose=0,
                                device=device)
entropy_duplicate_perf_hist = active_learning_procedure_duplicate(max_entropy,
                                           X_test,
                                           y_test,
                                           X_pool,
                                           y_pool,
                                           X_initial,
                                           y_initial,
                                           estimator,)
save_list(entropy_duplicate_perf_hist, "entropy_duplicate_perf_hist")

Accuracy after query 1: 0.6363
Accuracy after query 2: 0.6767
Accuracy after query 3: 0.6967
Accuracy after query 4: 0.7257
Accuracy after query 5: 0.7415
Accuracy after query 6: 0.6868
Accuracy after query 7: 0.6856
Accuracy after query 8: 0.6616
Accuracy after query 9: 0.7375
Accuracy after query 10: 0.7680
Accuracy after query 11: 0.8223
Accuracy after query 12: 0.7988
Accuracy after query 13: 0.8309
Accuracy after query 14: 0.8460
Accuracy after query 15: 0.8537
Accuracy after query 16: 0.8600
Accuracy after query 17: 0.8706
Accuracy after query 18: 0.8858
Accuracy after query 19: 0.8323
Accuracy after query 20: 0.8938
Accuracy after query 21: 0.9062
Accuracy after query 22: 0.8900
Accuracy after query 23: 0.9029
Accuracy after query 24: 0.9042
Accuracy after query 25: 0.9133
Accuracy after query 26: 0.9354
Accuracy after query 27: 0.9323
Accuracy after query 28: 0.9316
Accuracy after query 29: 0.9329
Accuracy after query 30: 0.9407
Accuracy after query 31: 0.9354
Accuracy after qu

# Ignore

In [21]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# estimator = NeuralNetClassifier(CNN,
#                                 max_epochs=50,
#                                 batch_size=128,
#                                 lr=0.001,
#                                 optimizer=torch.optim.Adam,
#                                 criterion=torch.nn.CrossEntropyLoss,
#                                 train_split=None,
#                                 verbose=0,
#                                 device=device)
# entropy_perf_hist = active_learning_procedure(max_entropy,
#                                               X_test,
#                                               y_test,
#                                               X_pool,
#                                               y_pool,
#                                               X_initial,
#                                               y_initial,
#                                               estimator,)

In [22]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# estimator = NeuralNetClassifier(CNN,
#                                 max_epochs=50,
#                                 batch_size=128,
#                                 lr=0.001,
#                                 optimizer=torch.optim.Adam,
#                                 criterion=torch.nn.CrossEntropyLoss,
#                                 train_split=None,
#                                 verbose=0,
#                                 device=device)
# bald_perf_hist = active_learning_procedure(bald,
#                                            X_test,
#                                            y_test,
#                                            X_pool,
#                                            y_pool,
#                                            X_initial,
#                                            y_initial,
#                                            estimator,)

In [23]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# estimator = NeuralNetClassifier(CNN,
#                                 max_epochs=50,
#                                 batch_size=128,
#                                 lr=0.001,
#                                 optimizer=torch.optim.Adam,
#                                 criterion=torch.nn.CrossEntropyLoss,
#                                 train_split=None,
#                                 verbose=0,
#                                 device=device)
# uniform_perf_hist = active_learning_procedure(uniform,
#                                               X_test,
#                                               y_test,
#                                               X_pool,
#                                               y_pool,
#                                               X_initial,
#                                               y_initial,
#                                               estimator,)

In [24]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()
# plt.plot(entropy_perf_hist, label="entropy")
# plt.plot(bald_perf_hist, label="bald")
# plt.plot(uniform_perf_hist, label="uniform")
# plt.ylim([0.7,1])
# plt.legend()